In [22]:

!git add PredictNHL.ipynb


!git commit -m "Add PredictNHL.ipynb"


!git pull origin main

!git push origin main

[main a31d2ee] Add PredictNHL.ipynb
 1 file changed, 167 insertions(+), 150 deletions(-)
Already up to date.


From https://github.com/nic01as1/NHLPredictor
 * branch            main       -> FETCH_HEAD
To https://github.com/nic01as1/NHLPredictor.git
   d2e6777..a31d2ee  main -> main


In [ ]:
#if stuck :
!git push origin main --force


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import time
import warnings
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score, log_loss, roc_auc_score, f1_score, classification_report, make_scorer)
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE

In [2]:
#AJOUTER LE NOMBRE DE JOURS DE REPOS ! et ajouter 2 autres mod`les (Neural et Logistic) 
#J'ai généralisé le modèle ,mais j'aimerai le rendre + spécific à la team. En ce moment j'utilise des variables différences par exempkle donc Rank Team home - rank team away par excemple

In [3]:
#Retrieve the rank of each teams

url = 'https://www.hockey-reference.com/leagues/NHL_2025.html#all_stats'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(2) 
    soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

table = soup.find('table', id='stats')

if table is None:
    print("Team Statistics table not found")
else:
    
    rows = table.find_all('tr')[1:] 
    teams = [
        row.find_all('td')[0].text.strip()
        for row in rows
        if row.find_all('td')
    ]
    ranks = list(range(1, len(teams) + 1)) 

    
    df_rank = pd.DataFrame({
        'Team': teams,
        'Rank': ranks,
        'Team Rank': [f"{team} {rank}" for team, rank in zip(teams, ranks)]
    })
    print(df_rank)
    

                     Team  Rank                 Team Rank
0           Winnipeg Jets     1           Winnipeg Jets 1
1     Washington Capitals     2     Washington Capitals 2
2            Dallas Stars     3            Dallas Stars 3
3        Florida Panthers     4        Florida Panthers 4
4    Vegas Golden Knights     5    Vegas Golden Knights 5
5     Toronto Maple Leafs     6     Toronto Maple Leafs 6
6     Tampa Bay Lightning     7     Tampa Bay Lightning 7
7         Edmonton Oilers     8         Edmonton Oilers 8
8     Carolina Hurricanes     9     Carolina Hurricanes 9
9      Colorado Avalanche    10     Colorado Avalanche 10
10      New Jersey Devils    11      New Jersey Devils 11
11         Minnesota Wild    12         Minnesota Wild 12
12      Los Angeles Kings    13      Los Angeles Kings 13
13  Columbus Blue Jackets    14  Columbus Blue Jackets 14
14      Detroit Red Wings    15      Detroit Red Wings 15
15      Vancouver Canucks    16      Vancouver Canucks 16
16          Bo

In [4]:

#Retrieve the infos of EACH games for the season

def scrape_game_history(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    if table is None:
        print(f"Table not found on {url}")
        return pd.DataFrame()  #

    
    data = {
        "Date": [],
        "Home Team": [],
        "Home Score": [],
        "Away Team": [],
        "Away Score": [],
        "Result": [],
        "Home Win": []
    }

    
    for row in table.find_all("tr")[1:]:  # Skip the header
        columns = row.find_all("td")
        if len(columns) < 6:  
            continue

        
        date = columns[1].text.strip()
        home_team = columns[3].text.strip()
        away_team = columns[4].text.strip()
        result = columns[5].text.strip()

        
        if "-" in result:  
            scores = result.split("-")
            if len(scores) == 2 and scores[0].strip().isdigit() and scores[1].strip().isdigit():
                home_score = int(scores[0].strip())
                away_score = int(scores[1].strip())
                home_win = home_score > away_score
            else:  
                home_score = away_score = home_win = None
        else:
            home_score = away_score = home_win = None

        
        data["Date"].append(date)
        data["Home Team"].append(home_team)
        data["Away Team"].append(away_team)
        data["Result"].append(result)
        data["Home Score"].append(home_score)
        data["Away Score"].append(away_score)
        data["Home Win"].append(home_win)

    
    df_game = pd.DataFrame(data)
    df_game["Date"] = df_game["Date"].apply(lambda x: x.split(" ")[0])  # Extract just the date
    return df_game



url_history = "https://fixturedownload.com/results/nhl-2024"


df_game = scrape_game_history(url_history)


if df_game.empty:
    print("No game data found.")
else:
    print(f"Scraped {len(df_game)} games.")
    print(df_game)

Scraped 1312 games.
            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1308  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   
1311  18/04/2025      Los Angeles Kings         NaN       Calgary Flames   

      Away Score Result Home Win  
0            4.0  1 - 4    False

In [5]:
#Normalise Both datasource
team_name_mapping = {
    "Montréal Canadiens": "Montreal Canadiens", 
    
}


df_game["Home Team"] = df_game["Home Team"].replace(team_name_mapping)
df_game["Away Team"] = df_game["Away Team"].replace(team_name_mapping)


In [6]:
# Merge df_rank to add Home Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Home Team", "Rank": "Home Team Rank"}), 
    on="Home Team", 
    how="left"
)

# Merge df_rank to add Away Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Away Team", "Rank": "Away Team Rank"}), 
    on="Away Team", 
    how="left"
)

# Reorder the final columns
final_columns = [
    "Date", "Home Team", "Home Score", "Away Team", "Away Score",
    "Home Team Rank", "Away Team Rank", "Home Win"
]
df_final = df_game.loc[:, final_columns] 

print(df_final)


            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres         1.0    New Jersey Devils   
1     05/10/2024      New Jersey Devils         3.0       Buffalo Sabres   
2     08/10/2024         Seattle Kraken         2.0      St. Louis Blues   
3     08/10/2024       Florida Panthers         6.0        Boston Bruins   
4     09/10/2024       Utah Hockey Club         5.0   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators         NaN  Carolina Hurricanes   
1308  17/04/2025       New York Rangers         NaN  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins         NaN  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets         NaN   New York Islanders   
1311  18/04/2025      Los Angeles Kings         NaN       Calgary Flames   

      Away Score  Home Team Rank  Away Team Rank Home Win  
0            4.0           

In [7]:
#DATA MANIPULATION


# Filter out the Unplayed Games
df_final = df_final.dropna(subset=["Home Score", "Away Score"])

# Convert Scores and Dates to Appropriate Data Types
df_final["Date"] = pd.to_datetime(df_final["Date"], format="%d/%m/%Y")
df_final[["Home Score", "Away Score"]] = df_final[["Home Score", "Away Score"]].astype(int)
df_final["Home Win"] = df_final["Home Win"].astype(bool)

# Sort Data by Date for Chronological Calculations
df_final = df_final.sort_values(by="Date").reset_index(drop=True)

# Function: Calculate Last 10 Games Stats
def calculate_last_10_stats(df, team_column):
    last_10_wins = []
    team_games = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        recent_games = team_games[team][-10:]  # Last 10 games
        last_10_wins.append(sum(recent_games))
        team_games[row["Home Team"]].append(row["Home Win"])
        team_games[row["Away Team"]].append(not row["Home Win"])
    return last_10_wins

# Add Last 10 Wins for Home and Away Teams
df_final["Home Last 10 Wins"] = calculate_last_10_stats(df_final, "Home Team")
df_final["Away Last 10 Wins"] = calculate_last_10_stats(df_final, "Away Team")

# Add Whether Teams Played Yesterday
def calculate_played_yesterday(df, team_column):
    played_yesterday = []
    last_game_date = {}

    for _, row in df.iterrows():
        team = row[team_column]
        played_yesterday.append(last_game_date.get(team) == row["Date"] - pd.Timedelta(days=1))
        last_game_date[team] = row["Date"]
    return played_yesterday

df_final["Home Played Yesterday"] = calculate_played_yesterday(df_final, "Home Team")
df_final["Away Played Yesterday"] = calculate_played_yesterday(df_final, "Away Team")

# Add Win Rate for Home and Away Teams
def calculate_win_rate(df, team_column, is_home_column):
    win_rate = []
    team_stats = {team: {"wins": 0, "games": 0} for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        stats = team_stats[team]
        win_rate.append(stats["wins"] / stats["games"] if stats["games"] > 0 else 0)
        
        
        if is_home_column:
            stats["wins"] += row["Home Win"]
        else:
            stats["wins"] += not row["Home Win"]
        stats["games"] += 1
    return win_rate

df_final["Home Win Rate"] = calculate_win_rate(df_final, "Home Team", is_home_column=True)
df_final["Away Win Rate"] = calculate_win_rate(df_final, "Away Team", is_home_column=False)

# Add Overall Win Streak for Home and Away Teams
def calculate_overall_win_streak(df):
    streak = {}
    home_streaks = []
    away_streaks = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]

        
        home_streaks.append(streak.get(home_team, 0))
        away_streaks.append(streak.get(away_team, 0))

        
        if row["Home Win"]:
            streak[home_team] = streak.get(home_team, 0) + 1
            streak[away_team] = 0
        else:
            streak[home_team] = 0
            streak[away_team] = streak.get(away_team, 0) + 1
    return home_streaks, away_streaks

home_streaks, away_streaks = calculate_overall_win_streak(df_final)
df_final["Home Team Overall Win Streak Before Game"] = home_streaks
df_final["Away Team Overall Win Streak Before Game"] = away_streaks

# Add Average Opponent Strength for Home and Away Teams
def calculate_avg_opponent_rank(df, team_column, opponent_rank_column):
    avg_opponent_rank = []
    opponent_stats = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        avg_opponent_rank.append(
            np.mean(opponent_stats[team][-10:]) if opponent_stats[team] else np.nan
        )
        opponent_stats[row["Home Team"]].append(row["Away Team Rank"])
        opponent_stats[row["Away Team"]].append(row["Home Team Rank"])
    return avg_opponent_rank

df_final["Home Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Home Team", "Away Team Rank")
df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Away Team", "Home Team Rank")

def calculate_days_since_last_game(df):
    last_game_date = {}  # Dictionary to track the last game date for each team
    home_days_since_last_game = []
    away_days_since_last_game = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        current_date = row["Date"]

        # Calculate days since last game for the home team
        if home_team in last_game_date:
            days_since_last_home_game = (current_date - last_game_date[home_team]).days
        else:
            days_since_last_home_game = None  
        home_days_since_last_game.append(days_since_last_home_game)

        # Calculate days since last game for the away team
        if away_team in last_game_date:
            days_since_last_away_game = (current_date - last_game_date[away_team]).days
        else:
            days_since_last_away_game = None  
        away_days_since_last_game.append(days_since_last_away_game)

        # Update the last game date for both teams
        last_game_date[home_team] = current_date
        last_game_date[away_team] = current_date

    return home_days_since_last_game, away_days_since_last_game


# Apply the function to the DataFrame
home_days, away_days = calculate_days_since_last_game(df_final)


df_final["Home Rest Days Since Last Game"] = home_days
df_final["Away Rest Days Since Last Game"] = away_days

print(df_final[["Date", "Home Team", "Away Team", "Home Opponent Strength", "Away Opponent Strength","Home Last 10 Wins","Away Last 10 Wins","Home Played Yesterday","Away Played Yesterday","Home Win Rate","Away Win Rate","Home Team Overall Win Streak Before Game","Away Team Overall Win Streak Before Game","Away Rest Days Since Last Game","Home Rest Days Since Last Game"]].tail(10))


df_final.to_csv("enhanced_game_data.csv", index=False)


          Date              Home Team            Away Team  \
949 2025-03-01       Florida Panthers       Calgary Flames   
950 2025-03-01  Columbus Blue Jackets    Detroit Red Wings   
951 2025-03-02        St. Louis Blues    Los Angeles Kings   
952 2025-03-02       Utah Hockey Club    New Jersey Devils   
953 2025-03-02          Winnipeg Jets  Philadelphia Flyers   
954 2025-03-02          Anaheim Ducks   Chicago Blackhawks   
955 2025-03-02        Ottawa Senators      San Jose Sharks   
956 2025-03-02         Buffalo Sabres   Montreal Canadiens   
957 2025-03-02    Carolina Hurricanes      Edmonton Oilers   
958 2025-03-02         Seattle Kraken    Vancouver Canucks   

     Home Opponent Strength  Away Opponent Strength  Home Last 10 Wins  \
949                    17.9                    15.6                  7   
950                    16.6                    15.6                  6   
951                    13.2                    11.6                  5   
952                  

In [25]:
df_final["Rank Difference"] = df_final["Home Team Rank"] - df_final["Away Team Rank"]
df_final["Home Advantage"] = df_final["Home Win Rate"] - df_final["Away Win Rate"]
df_final["Win Streak Impact"] = df_final["Home Team Overall Win Streak Before Game"] - df_final["Away Team Overall Win Streak Before Game"]
df_final["Opponent Strength"] = df_final["Home Opponent Strength"] - df_final["Away Opponent Strength"]
df_final["Last 10 Wins"] = df_final["Home Last 10 Wins"] - df_final["Away Last 10 Wins"]
df_final["Opponent Strength"] = df_final["Home Opponent Strength"] - df_final["Away Opponent Strength"]

In [26]:

results1 = df_final
results1.tail(10)

,Date,Home Team,Home Score,Away Team,Away Score,Home Team Rank,Away Team Rank,Home Win,Home Last 10 Wins,Away Last 10 Wins,...,Away Team Overall Win Streak Before Game,Home Opponent Strength,Away Opponent Strength,Home Rest Days Since Last Game,Away Rest Days Since Last Game,Rank Difference,Home Advantage,Win Streak Impact,Opponent Strength,Last 10 Wins
949,2025-03-01,Florida Panthers,3,Calgary Flames,0,4,20,True,7,4,...,0,17.9,15.6,1.0,1.0,-16,0.192593,2,2.3,3
950,2025-03-01,Columbus Blue Jackets,5,Detroit Red Wings,3,14,15,True,6,7,...,0,16.6,15.6,1.0,1.0,-1,0.136654,3,1.0,-1
951,2025-03-02,St. Louis Blues,4,Los Angeles Kings,1,18,13,True,5,5,...,0,13.2,11.6,2.0,1.0,5,0.076613,2,1.6,0
952,2025-03-02,Utah Hockey Club,1,New Jersey Devils,3,22,11,False,6,5,...,0,15.3,19.4,2.0,3.0,11,-0.148387,3,-4.1,1
953,2025-03-02,Winnipeg Jets,1,Philadelphia Flyers,2,1,24,False,9,4,...,0,19.3,17.0,2.0,2.0,-23,0.387097,0,2.3,5
954,2025-03-02,Anaheim Ducks,3,Chicago Blackhawks,6,26,31,False,7,2,...,0,19.2,12.3,2.0,2.0,-5,0.300000,1,6.9,5
955,2025-03-02,Ottawa Senators,5,San Jose Sharks,3,19,32,True,5,1,...,0,11.2,17.1,3.0,2.0,-13,0.399044,0,-5.9,4
956,2025-03-02,Buffalo Sabres,2,Montreal Canadiens,4,29,21,False,6,4,...,3,18.8,16.2,2.0,2.0,8,0.035714,-3,2.6,2
957,2025-03-02,Carolina Hurricanes,1,Edmonton Oilers,3,9,8,False,4,3,...,0,18.3,14.5,2.0,2.0,1,0.206897,1,3.8,1
958,2025-03-02,Seattle Kraken,6,Vancouver Canucks,3,28,16,True,3,5,...,0,15.6,16.2,4.0,2.0,12,-0.096774,0,-0.6,-2


In [21]:

scorer = make_scorer(f1_score, pos_label=None, average='weighted')

# Define y_true based on the Home Win column
y_true = df_final["Home Win"].astype(int)  # Convert True/False to 1/0

# Create a dummy prediction where all predictions are 1 (Home Win = True)
y_pred = [1] * len(y_true)

# Calculate the weighted F1 score
f1 = f1_score(y_true, y_pred, pos_label=None, average="weighted")

# Print the F1 score
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.3861


In [23]:

#Target and features
target = "Home Win"
features = [
    "Opponent Strength",
    "Rank Difference",
    "Last 10 Wins",
    "Home Played Yesterday",
    "Away Played Yesterday",
    "Home Advantage",
    "Win Streak Impact",
    "Away Rest Days Since Last Game",
    "Home Rest Days Since Last Game"
]


X = df_final[features]
y = df_final[target].astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

#Param_Grid
param_grid_rf = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

param_grid_dt = {
    "max_depth": [5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}


rf = RandomForestClassifier(random_state=42)
dt = DecisionTreeClassifier(random_state=42)


grid_search_rf = GridSearchCV(rf, param_grid_rf, scoring="f1_weighted", cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_
print(f"Best Random Forest Params: {grid_search_rf.best_params_}")


grid_search_dt = GridSearchCV(dt, param_grid_dt, scoring="f1_weighted", cv=5, n_jobs=-1)
grid_search_dt.fit(X_train, y_train)
best_dt = grid_search_dt.best_estimator_
print(f"Best Decision Tree Params: {grid_search_dt.best_params_}")


models = {
    "Random Forest": best_rf,
    "Decision Tree": best_dt
}


results = []

#Loop Train of models
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average="weighted")
    results.append({"Model": model_name, "F1 Score": f1})

    # Feature importance
    if hasattr(model, "feature_importances_"):
        feature_importance_df = pd.DataFrame({
            "Feature": features,
            "Importance": model.feature_importances_
        }).sort_values(by="Importance", ascending=False)
        
        print(f"\nFeature Importance ({model_name}):")
        print(feature_importance_df)
        feature_importance_df.to_csv(f"feature_importance_{model_name.lower().replace(' ', '_')}.csv", index=False)

    #PErmutation
    perm_importance = permutation_importance(model, X_test, y_test, scoring="f1_weighted", random_state=42)
    perm_importance_df = pd.DataFrame({
        "Feature": features,
        "Importance": perm_importance.importances_mean
    }).sort_values(by="Importance", ascending=False)

    print(f"\nPermutation Importance ({model_name}):")
    print(perm_importance_df)
    perm_importance_df.to_csv(f"feature_importance_permutation_{model_name.lower().replace(' ', '_')}.csv", index=False)


results_df = pd.DataFrame(results).sort_values(by="F1 Score", ascending=False)
print("\nModel Performance Summary:")
print(results_df)

# Save model performance results
results_df.to_csv("model_performance_comparison.csv", index=False)

Best Random Forest Params: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Decision Tree Params: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}

Feature Importance (Random Forest):
                          Feature  Importance
1                 Rank Difference    0.394923
5                  Home Advantage    0.179857
0               Opponent Strength    0.170023
2                    Last 10 Wins    0.076216
6               Win Streak Impact    0.071158
8  Home Rest Days Since Last Game    0.043101
7  Away Rest Days Since Last Game    0.042897
4           Away Played Yesterday    0.015727
3           Home Played Yesterday    0.006099

Permutation Importance (Random Forest):
                          Feature  Importance
1                 Rank Difference    0.034057
8  Home Rest Days Since Last Game    0.017583
7  Away Rest Days Since Last Game    0.002607
2                    Last 10 Wins    0.002127
3           Home Played Yeste

In [27]:
# Retain only the important features with positive permutation importance for RF which is the best model

selected_features = [
    "Rank Difference",  
    "Home Rest Days Since Last Game", "Away Rest Days Since Last Game",
    "Last 10 Wins"
]




In [28]:
# Initialize Label Encoder
encoder = LabelEncoder()


df_final["Home Team Encoded"] = encoder.fit_transform(df_final["Home Team"])
df_final["Away Team Encoded"] = encoder.transform(df_final["Away Team"])

print(df_final[["Home Team", "Home Team Encoded"]].head())
print(df_final[["Away Team", "Away Team Encoded"]].head())


encoder.fit(results1["Home Team"].values)

home_teams = encoder.transform(results1["Home Team"].values)
visitor_teams = encoder.transform(results1["Away Team"].values)


X_teams = np.vstack([home_teams, visitor_teams]).T
print(X_teams[:5], X_teams.shape)  # Check first few rows & dimensions


onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).toarray()

print(X_teams[:5]) 
print(X_teams.shape)  



           Home Team  Home Team Encoded
0     Buffalo Sabres                  2
1  New Jersey Devils                 16
2     Seattle Kraken                 23
3   Florida Panthers                 11
4   Utah Hockey Club                 27
            Away Team  Away Team Encoded
0   New Jersey Devils                 16
1      Buffalo Sabres                  2
2     St. Louis Blues                 24
3       Boston Bruins                  1
4  Chicago Blackhawks                  5
[[ 2 16]
 [16  2]
 [23 24]
 [11  1]
 [27  5]] (959, 2)
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [29]:


X_homehigher = df_final[selected_features].values 
X_all = np.hstack([X_homehigher, X_teams])
print("Feature matrix shape:", X_all.shape)


Feature matrix shape: (959, 68)


In [30]:
X_homehigher = df_final[selected_features].values 

print("Shape of Numerical Features (X_homehigher):", X_homehigher.shape)
print("Shape of One-Hot Encoded Teams (X_teams):", X_teams.shape)


X_all = np.hstack([X_homehigher, X_teams])
print(" Final Feature Matrix Shape (X_all):", X_all.shape)


sample_data = pd.DataFrame(X_all)


print("\n Preview of last 5 observations:")
print(sample_data.tail())



Shape of Numerical Features (X_homehigher): (959, 4)
Shape of One-Hot Encoded Teams (X_teams): (959, 64)
 Final Feature Matrix Shape (X_all): (959, 68)

 Preview of last 5 observations:
       0    1    2    3    4    5    6    7    8    9   ...   58   59   60  \
954  -5.0  2.0  2.0  5.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
955 -13.0  3.0  2.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
956   8.0  2.0  2.0  2.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
957   1.0  2.0  2.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
958  12.0  4.0  2.0 -2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

      61   62   63   64   65   66   67  
954  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
955  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
956  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
957  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
958  0.0  0.0  0.0  1.0  0.0  0.0  0.0  

[5 rows x 68 columns]


In [31]:
#RANDOM FOREST
param_grid_rf = {
    "max_features": ['sqrt', 'log2', 0.1],
    "n_estimators": [100, 200, 300],  
    "criterion": ["gini",'entropy'],  
    "min_samples_leaf": [1, 2, 4],  
    "max_depth": [20, 30],  
    "min_samples_split": [2, 5],  
    "bootstrap": [True]  
}


scorer = make_scorer(f1_score, average="weighted")


rf_model = RandomForestClassifier(random_state=10)


print("\nTraining Random Forest with Grid Search...")
grid_rf = GridSearchCV(rf_model, param_grid_rf, scoring=scorer, error_score="raise", cv=5, n_jobs=-1)


try:
    grid_rf.fit(X_all, y_true)  
    best_rf_model = grid_rf.best_estimator_  

    print("\nBest Random Forest Model:")
    print(f"   - Best F1 Score: {grid_rf.best_score_:.4f}")
    print(f"   - Best Parameters: {grid_rf.best_params_}")

except Exception as e:
    print(f"Error during Grid Search for Random Forest: {e}")
    exit()



Training Random Forest with Grid Search...

Best Random Forest Model:
   - Best F1 Score: 0.6038
   - Best Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}


In [32]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Step 1: Open Chrome and fetch NHL odds
with webdriver.Chrome() as driver:
    url = "https://betway.com/en/sports/grp/ice-hockey/north-america/nhl"
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

driver.quit()

# Step 2: Extract match odds
matches_data = []
games = soup.find_all(class_="oneLineEventItem")

for game in games:
    try:
        teams = game.find_all(class_="teamNameFirstPart")
        odds = game.find_all(class_="odds")

        if len(teams) >= 2 and len(odds) >= 2:
            matches_data.append({
                "Away Team": teams[0].text.strip(),
                "Away Odds": float(odds[0].text.strip()),
                "Home Team": teams[1].text.strip(),
                "Home Odds": float(odds[1].text.strip())
            })
    except Exception as e:
        print(f"Error extracting data: {e}")

df_odds = pd.DataFrame(matches_data)
print(df_odds)

# Step 3: Automated Prediction for All Games
predictions = []

              Away Team  Away Odds             Home Team  Home Odds
0   Toronto Maple Leafs       1.16   Pittsburgh Penguins       5.00
1         Boston Bruins       2.30        Minnesota Wild       1.62
2        Calgary Flames       3.60   Carolina Hurricanes       1.30
3        St Louis Blues       2.85          Dallas Stars       1.44
4   Nashville Predators       2.45      New York Rangers       1.57
5     New Jersey Devils       2.35  Vegas Golden Knights       1.60
6       Ottawa Senators       2.30   Washington Capitals       1.62
7   Tampa Bay Lightning       2.15      Florida Panthers       1.74
8        Buffalo Sabres       2.00    Montreal Canadiens       1.83
9    New York Islanders       2.10      New York Rangers       1.76
10      San Jose Sharks       4.00   Toronto Maple Leafs       1.25
11             LA Kings       1.42    Chicago Blackhawks       3.00


In [42]:
# best_rf_model = grid_rf.best_estimator_

# Display available teams
teams = list(encoder.classes_)  # Ensure encoder.classes_ is pre-loaded

bankroll = float(input("Enter your total bankroll: "))  # User-defined bankroll

predictions = []  # List to store predictions

# Store bet amounts for scaling
total_bet_amount = 0
bet_amounts = []

for index, row in df_odds.iterrows():
    home_team = row["Home Team"]
    away_team = row["Away Team"]
    home_team_odds = row["Home Odds"]
    away_team_odds = row["Away Odds"]

    # Check if teams are in the encoder
    if home_team not in teams or away_team not in teams:
        print(f"Skipping {home_team} vs {away_team} due to missing team data.")
        continue

    # Directly use the selected features without needing to distinguish between home/away
    all_features = selected_features  # Use the full list of features

    # Extract the stats for the selected features for home and away teams
    home_team_stats = results1[results1["Home Team"] == home_team][all_features].mean()
    away_team_stats = results1[results1["Away Team"] == away_team][all_features].mean()

    # Check for missing data
    if home_team_stats.isnull().any() or away_team_stats.isnull().any():
        print(f"Skipping {home_team} vs {away_team} due to insufficient season data.")
        continue

    # Prepare input features
    numerical_features = np.hstack([home_team_stats.values, away_team_stats.values])
    num_teams_in_training = best_rf_model.n_features_in_ - len(numerical_features)
    
    # One-hot encoding for home and away teams
    one_hot_home = [1 if team == home_team else 0 for team in teams][:num_teams_in_training // 2]
    one_hot_away = [1 if team == away_team else 0 for team in teams][:num_teams_in_training // 2]

    if len(numerical_features) + len(one_hot_home) + len(one_hot_away) != best_rf_model.n_features_in_:
        print(f"Feature mismatch for {home_team} vs {away_team}. Skipping prediction.")
        continue

    # Input Vector
    input_vector = np.hstack([numerical_features, one_hot_home, one_hot_away]).reshape(1, -1)

    # Predict probabilities
    predicted_prob = best_rf_model.predict_proba(input_vector)
    home_team_prob = predicted_prob[0][1]
    away_team_prob = predicted_prob[0][0]

    # Kelly Criterion Calculation
    def kelly_criterion(prob, odds):
        b = odds - 1
        q = 1 - prob
        return max(0, (b * prob - q) / b)

    home_kelly = kelly_criterion(home_team_prob, home_team_odds)
    away_kelly = kelly_criterion(away_team_prob, away_team_odds)

    # Store the bet amounts before scaling
    home_bet = home_kelly * bankroll
    away_bet = away_kelly * bankroll

    # Add these to total bet amounts
    total_bet_amount += home_bet + away_bet
    bet_amounts.append((home_bet, away_bet))

    # Store raw values for later scaling
    predictions.append({
        "Home Team": home_team,
        "Away Team": away_team,
        "Home Probability": home_team_prob,
        "Away Probability": away_team_prob,
        "Home Kelly": home_kelly,
        "Away Kelly": away_kelly
    })

# Scale bets if total bet amount exceeds bankroll
if total_bet_amount > bankroll:
    scaling_factor = bankroll / total_bet_amount
    print(f"Scaling bets by a factor of {scaling_factor:.2f} to stay within bankroll.")

    for i, (home_bet, away_bet) in enumerate(bet_amounts):
        scaled_home_bet = round(home_bet * scaling_factor)
        scaled_away_bet = round(away_bet * scaling_factor)

        # Update predictions with scaled values
        predictions[i]["Home Bet"] = scaled_home_bet
        predictions[i]["Away Bet"] = scaled_away_bet

        # Update bankroll with scaled bet amounts
        bankroll -= (scaled_home_bet + scaled_away_bet)

else:
    print("Total bet amount is within bankroll. No scaling required.")

# Save predictions with the final scaled bet amounts
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")


Enter your total bankroll:  77


Skipping Dallas Stars vs St Louis Blues due to missing team data.
Skipping Chicago Blackhawks vs LA Kings due to missing team data.
Scaling bets by a factor of 0.72 to stay within bankroll.
Predictions saved to predictions.csv
